Ben Trevett 의 [Sequence to Sequence Learning with Neural Networks](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 [AI Hub 한국어-영어 번역 말뭉치](http://www.aihub.or.kr/aidata/87/download)를 이용한다.

이 모델에서는 [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)의 구조대로 간단한 모델을 만들어본다.

# 개요

가장 유명한 seq2seq 모델은 RNN을 이용한 인코더-디코더 모델이다. 입력 문장은 인코더를 통해 하나의 문맥 벡터로 변환되고, 이후 디코더에서 타겟 문장으로 변환된다.

<img src = 'https://github.com/bentrevett/pytorch-seq2seq/raw/49cbdd39d934633ab69b7ff0cf4ef0da33a42e18/assets/seq2seq1.png'>

문장의 처음에 `<sos>`, 마지막에 `<eos>` 토큰을 추가하여 입력으로 넣는다. 입력 문장의 각 단어 $x_t$에 대해 벡터 임베딩을 $e(x_t)$, 이전 타임의 히든 벡터를 $h_{t-1}$ 이라 하면 인코더는 다음과 같이 정의된다.

$$ h_t = \text{EncoderRNN}(e(X_t), h_{t-1}) $$

여기서 $X = \{ x_1,\ldots, x_T\}$, $x_1 = <sos>$ 등이다. 마지막 단어에 대한 히든 벡터 $h_T$를 문장의 벡터로 정의한다.

디코더는 타겟 문장 $(y_t)$의 임베딩 $d(y_t)$와 이전 히든 벡터 $s_{t-1}$에 대해 다음과 같이 정의된다($s_0 = h_T$).

$$ s_t = \text{DecoderRNN}(d(y_t), s_{t-1}) $$

마지막으로 디코더의 출력을 선형 레이어 $f$에 태워 각 단어의 최종 스코어 $\hat{y}_t = f(s_t)$ 를 만든다. 타겟 문장의 입력은 항상 `<sos>` 토큰으로 시작하지만, 그 이후의 벡터는 실제 타겟 단어 $y_t$를 입력을 넣을 수도 있고 아니면 이전 단계의 출력 $\hat{y}_t$를 넣을 수도 있다. 

이렇게 얻은 타겟 벡터 $\hat{Y} = \{ \hat{y}_1, \ldots, \hat{y}_T\}$ 를 실제 정답과 비교하여 손실을 계산하고 모델을 업데이트한다.

# 전처리

이후 사용할 모듈들을 불러오자.

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
from konlpy.tag import Mecab
import numpy as np

import random
import math
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

랜덤 시드를 고정하자.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

토크나이저를 지정하자. 여기서 한글은 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢, 영어는 [spaCy](https://spacy.io/)를 이용한다.

In [3]:
mecab = Mecab() # 한글
spacy_en = spacy.load('en') # 영어

토크나이저 함수를 만들자. 논문에 따르면 입력 문장은 역순으로 넣는 게 좋다고 하니 그대로 따르자.

In [6]:
def tokenize_ko(text):
    return [tok for tok in mecab.morphs(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [7]:
tokenize_ko('한글은 너무 어렵당!')

['!', '당', '어렵', '너무', '은', '한글']

In [8]:
tokenize_en('Korean is too dificult for me!')

['Korean', 'is', 'too', 'dificult', 'for', 'me', '!']

TorchText의 `Field`를 이용해서 데이터 입력 포맷을 지정하자. `init_token`과 `eos_token` 인자를 이용해서 문장 처음과 마지막에 `<sos>`와 `<eos>` 토큰을 자동으로 추가할 수 있다.

In [9]:
SRC = Field(tokenize = tokenize_ko,
           init_token = '<sos>',
           eos_token = '<eos>',
           )

TRG = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True
           )

In [10]:
fields = {'ko': ('ko',SRC), 'en': ('en',TRG)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [12]:
!ls data

1.구어체.xlsx  3.문어체-뉴스.xlsx  train_data.csv
2.대화체.xlsx  test_data.csv	   valid_data.csv


In [18]:
train_data, test_data = TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
valid_data = TabularDataset(path = 'data/valid_data.csv',
                            format = 'csv',
                            fields = fields,  )

불러온 데이터는 다음과 같은 형태이다.

In [19]:
vars(train_data[0]), vars(valid_data[0])

({'ko': ['.', '요', '가', '안', '가', '이해', '이', '문장', '이', '님', '선생'],
  'en': ['sir',
   ',',
   'i',
   'do',
   "n't",
   'understand',
   'this',
   'sentence',
   'here',
   '.']},
 {'ko': ['.', '가요', '로', '기숙사', '자마자', '끝나', '가', '학교'],
  'en': ['i',
   'go',
   'to',
   'dormitory',
   'as',
   'soon',
   'as',
   'i',
   'finished',
   'class',
   '.']})

길이를 확인해보자.

In [20]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 94463
Number of validation examples: 31688
Number of testing examples: 31822


이제 단어장을 만들자. `min_freq` 옵션을 이용하여 최소 2번 이상 등장하는 단어만 사용하도록 하자. 또한 단어장은 검증/테스트셋은 써서는 안된다.

In [21]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [22]:
print(f"Unique tokens in source (ko) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (ko) vocabulary: 33345
Unique tokens in target (en) vocabulary: 24619


이터레이터를 만들자. 일반적인 `Iterator` 대신 `BucketIterator` 쓰면 입력/출력 배치 안에 있는 문장의 길이가 최대한 비슷하게 되어 패딩을 최소화하게 해준다.

In [25]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

# Seq2seq 모델 생성

인코더, 디코더, 그리고 seq2seq 모델 순서대로 만들자.

## 인코더

인코더는 2레이어 LSTM을 사용하여 다음과 같이 정의한다.

$$ (h_t^1,c_t^1)= \text{EncoderLSTM}^1(e(x_t),(h_{t-1}^1,c_{t-1}^1)) $$

$$ (h_t^2,c_t^2)= \text{EncoderLSTM}^2(h_t^1,(h_{t-1}^2, c_{t-1}^2)) $$

<img src = 'https://github.com/bentrevett/pytorch-seq2seq/raw/49cbdd39d934633ab69b7ff0cf4ef0da33a42e18/assets/seq2seq2.png'>

인코더는 다음 인수들을 입력으로 받는다.

* `input_dim` : 입력 문장의 단어 갯수
* `emb_dim` : 임베딩 차원
* `hid_dim` : 히든 차원
* `n_layers` : RNN 계층수
* `dropout` : dropout 비율. 계층 사이에 적용된다.